In [ ]:
# ! pip install google-cloud-translate pandas requests bs4 google-generativeai

In [1]:
# Lưu vào file csv
import requests
import time
import os
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import google.generativeai as genai
import pandas as pd

In [ ]:
# ggsearchAPI = os.environ.get('GGSEARCH_API_KEY')
# ggsearchCSE = os.environ.get('GGSEARCH_CSE_ID')

ggsearchAPI = os.getenv('GGSEARCH_API_KEY')
ggsearchCSE = os.getenv('GGSEARCH_CSE_ID')
api_key = ggsearchAPI   # Thay thế bằng API key của bạn
cse_id = ggsearchCSE    # Thay thế bằng Custom Search Engine ID của bạn

base_path = os.path.join(os.getcwd(), 'Medicines-Data-Imputation/PreData/RAW_vn_medicines_202405171011.csv')
global missing_imputation
missing_imputation_and_standard = os.path.join(os.getcwd(), 'Medicines-Data-Imputation/DataOut/missing_activeIngredient_imputation_and_standard_unit_vn.csv')

# Hàm tìm kiếm Google và lấy thông tin dạng bào chế
def search_google(query, api_key, cse_id):
    query_params = {
        'q': f"{query} site:thuocbietduoc.com.vn",
        'key': api_key,
        'cx': cse_id
    }
    url = f"https://www.googleapis.com/customsearch/v1?{urlencode(query_params)}"
  
    retries = 5
    for i in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Ném lỗi HTTPError cho các phản hồi xấu

            if response.status_code == 200:
                results = response.json()
                
                if 'items' in results and results['items']:
                    first_result = results['items'][0]
                    link = first_result['link']
                    
                    response_detail = requests.get(link)
                    response_detail.raise_for_status()
                    
                    if response_detail.status_code == 200:
                        soup_detail = BeautifulSoup(response_detail.text, 'html.parser')
                        all_spans = soup_detail.find_all('span')
                        for span in all_spans:
                            if 'Dạng bào chế:' in span.get_text():
                                dosage_form = span.find_next('span', class_='textdetaildrg')
                                if dosage_form:
                                    return dosage_form.get_text()
                        return None
                else:
                    return None
            elif response.status_code == 429:
                wait_time = 2 ** i
                print(f"Đã bị giới hạn. Chờ {wait_time} giây.")
                time.sleep(wait_time)
            else:
                return None
        
        except requests.exceptions.RequestException as e:
            print(f"Lỗi yêu cầu: {e}")
            if i < retries - 1:  # Nếu không phải lần thử cuối cùng, chờ và thử lại
                wait_time = 2 ** i
                print(f"Chờ {wait_time} giây trước khi thử lại.")
                time.sleep(wait_time)
            else:
                return None

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv(base_path)

# Duyệt qua các hàng của DataFrame và tìm kiếm thông tin dạng bào chế nếu ô trống
for index, row in df.iterrows():
    if pd.isna(row['form']):
        medicine_name = row['name']  # Giả sử cột tên thuốc là 'name'
        dosage_form = search_google(medicine_name, api_key, cse_id)
        if dosage_form:
            df.at[index, 'form'] = dosage_form

# Lưu lại DataFrame vào tệp CSV mới
output_file_path = missing_imputation_and_standard
df.to_csv(output_file_path, index=False)

In [ ]:
# Khởi tạo Gemini API
# geminiAPI = os.environ.get("GEMINI_API_KEY")

geminiAPI = os.getenv("GEMINI_API_KEY") 
genai.configure(api_key=geminiAPI)

generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 2048,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  }
]

gemini = genai.GenerativeModel(model_name="gemini-1.5-flash",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

def standardize_unit(value):
  try:
    prompt = f"Bạn là một trợ lý AI có khả năng chuẩn hóa đơn vị của thông tin thuốc. \
    Chuẩn hóa đơn vị sau thành mg, ml, hoặc mg/ml(Lưu ý chỉ trả về kết quả trên 1 dòng): {value}"
    response = gemini.generate_content(prompt)
    return response.text.strip()
  except Exception as e:
    print(f"Lỗi khi gọi API: {e}")
    return ""

def is_standard_unit(value):
  if not value or value == "nan" or pd.isnull(value):
    return True
  units = ["mg", "ml", "mg/ml"]
  for unit in units:
    value = str(value).lower()
    if unit in value:
      return True
  return False

# Đọc file CSV
df = pd.read_csv(missing_imputation_and_standard)

# Chuẩn hóa đơn vị trong cột "strength"
for index, row in df.iterrows():
    strength = row['strength']
    if not is_standard_unit(strength):
        time.sleep(0.1)
        standardized_strength = standardize_unit(strength)
        if standardized_strength:
            df.at[index, 'strength'] = standardized_strength
            # print(f"Đã chuẩn hóa: {strength} -> {standardized_strength}")

df.to_csv(missing_imputation_and_standard, index=False)